# 1. **Setup**

#### ***Install Groq Python API library*** :

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 4.8 MB/s eta 0:00:00


#### ***Import Dependencies*** :

In [ ]:
# Standard library imports
import json
import os
from typing import Dict, List, Optional, Any

# Third-party imports
import requests
from bs4 import BeautifulSoup
from google.colab import userdata # Given
from groq import Groq
from IPython.display import Markdown, display, update_display

# 2. **API Key**

In [ ]:
groq_api_key = userdata.get('Groq_API_Key') # The 'Name' of your API key
groq = Groq(api_key=groq_api_key) # 'qroq' is for general access to the model when using its API for E.g responses.

#### ***Define LLM Model*** :

- Models available at Groq - https://console.groq.com/docs/models

In [ ]:
MODEL = "llama3-70b-8192" # wow

# 3. **Scrape Website using BeautifulSoup**

- Scrape a website using BeautifulSoup to retrieve all 'links'

In [ ]:
# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that has been scraped.

    This class fetches a webpage, parses its content, extracts the title,
    text content, and all links found on the page.

    Attributes:
        url (str): The URL of the webpage.
        body (bytes): The raw HTML content of the webpage.
        title (str): The title of the webpage or "No title found" if none exists.
        text (str): The text content of the webpage with scripts and styles removed.
        links (list): A list of all href attributes found in anchor tags.
    """

    url: str
    body: bytes
    title: str
    text: str
    links: List[str]

    def __init__(self, url: str) -> None:
        """
        Initialize a Website object by fetching and parsing the webpage.

        Args:
            url (str): The URL of the webpage to fetch and parse.
        """
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self) -> str:
        """
        Get a formatted string with the webpage title and content.

        Returns:
            str: A formatted string containing the webpage title and text content.
        """
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
huggingface = Website("https://huggingface.co/")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/sesame/csm-1b',
 '/google/gemma-3-27b-it',
 '/mistralai/Mistral-Small-3.1-24B-Instruct-2503',
 '/Qwen/QwQ-32B',
 '/deepseek-ai/DeepSeek-R1',
 '/models',
 '/spaces/sesame/csm-1b',
 '/spaces/Wan-AI/Wan2.1',
 '/spaces/Qwen/QwQ-32B-Demo',
 '/spaces/jasperai/LBM_relighting',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/open-r1/codeforces-cots',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/facebook/natural_reasoning',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Write

# 4. **Select Relevant Links using LLM**

- Make an **API call** to Groq to read the links from scraped webpage -> respond in structured **JSON**.
  
- ***LLM should decide which links are relevant***

- Then, LLM will replace relative links such as "/about" with "https://company.com/about"

- Excellent use case for LLM - nuanced (微妙的) understanding.

- No needs to parse (分析) and analyze the webpage yourself.

#### ***System Prompt*** :
- System prompt acts like an **instruction manual** that tells AI how to behave.

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



#### ***User Prompt*** :
- User prompt is like a question or request you hand to an AI.

In [ ]:
def get_links_user_prompt(website: 'Website') -> str:
    """
    This function generates a user prompt that contains all the links found on
    a given website.

    Args:
        website (Website): A Website object containing scraped website data.

    Returns:
        str: A formatted string containing instructions and the list of links
             from the website.
    """
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, \
                    respond with the full https URL in JSON format. \
                    Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
print(get_links_user_prompt(huggingface))

Here is the list of links on the website of https://huggingface.co/ - please decide which of these are relevant web links for a brochure about the company,                     respond with the full https URL in JSON format.                     Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/models
/datasets
/spaces
/posts
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/sesame/csm-1b
/google/gemma-3-27b-it
/mistralai/Mistral-Small-3.1-24B-Instruct-2503
/Qwen/QwQ-32B
/deepseek-ai/DeepSeek-R1
/models
/spaces/sesame/csm-1b
/spaces/Wan-AI/Wan2.1
/spaces/Qwen/QwQ-32B-Demo
/spaces/jasperai/LBM_relighting
/spaces/Kwai-Kolors/Kolors-Virtual-Try-On
/spaces
/datasets/FreedomIntelligence/medical-o1-reasoning-SFT
/datasets/open-r1/codeforces-cots
/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k
/datasets/facebook/natural_reasoning
/datasets/open-r1/OpenR1-Math-220k
/datasets
/join
/pricing#endpoints
/pricing#spaces
/pricing
/enterprise

#### ***API Call to Groq*** :

- Combining `link_system_prompt()` & `get_links_user_prompt()` from above.

- Now you can ask LLM to help you select relevant links.

- Compared with all the links you saw in Section 3, LLM has filerted the long list to just a few links!

In [ ]:
def get_links(url: str) -> Dict[str, Any]:
    """
    Fetches a webpage and extracts relevant links using an LLM.

    This function creates a Website object from the provided URL,
    then uses the Groq API to analyze the links on the page and
    return only those relevant for a company brochure.

    Args:
        url(str): The URL of the webpage to analyze.

    Returns:
        A dictionary containing the filtered relevant links parsed from JSON.
    """
    website = Website(url)
    response = groq.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
get_links("https://huggingface.co/")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/brand'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'}]}

# 5. **Scraping Relevant Links**

- Use BeautifulSoup to scrape the ( LLM produced ) relevant links.

- Don't panic if you see a long list of content. These are all the contents found in the relevant links.

In [ ]:
def get_all_details(url: str) -> str:
    """
    Fetches and combines content from a main webpage and its relevant linked pages.

    This function creates a comprehensive report by:
    1. Scraping the main URL provided
    2. Finding relevant links on that page using the get_links function
    3. Scraping each relevant linked page
    4. Combining all content into a single formatted string

    Args:
        url(str): The URL of the main webpage to analyze.

    Returns:
        A formatted string containing the main page content followed by
        content from all relevant linked pages.
    """
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/brand'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
sesame/csm-1b
Updated
2 days ago
•
7.62k
•
1.15k
google/gemma-3-27b-it
Updated
6 days ago
•
302k
•
761
mistralai/Mistral-Small-3.1-24B-Instruct-2503
Updated
19 minutes ago
•
476
Qwen/QwQ-32B
Updated
7 days ago
•
447k
•
2.34k
deepseek-ai/DeepSeek-R1
Updated
22 days ago
•
1.84M
•
11.4k
Browse 1M+ models
Spaces
Running
on
Zero
46

# 6. **Company Brochure**

- With the scraped website contents from the relevant links

- We can give it to LLM and let it generate a Company Brocure.

#### ***System Prompt*** :

In [ ]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. USE AS MANY EMOJI AS POSSIBLE"


#### ***User Prompt*** :

In [ ]:
def get_brochure_user_prompt(company_name: str, url: str) -> str:
    """
    Creates a prompt for generating a company brochure in markdown.

    Parameters:
        company_name(str): Name of the company to create a brochure for.
        url(str): URL of the company's website.

    Returns:
        A prompt string
    """
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000]  # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("Hugging Face", "https://huggingface.co/")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}]}


'You are looking at a company called: Hugging Face\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nsesame/csm-1b\nUpdated\n2 days ago\n•\n7.62k\n•\n1.15k\ngoogle/gemma-3-27b-it\nUpdated\n6 days ago\n•\n302k\n•\n761\nmistralai/Mistral-Small-3.1-24B-Instruct-2503\nUpdated\n19 minutes ago\n•\n476\nQwen/QwQ-32B\nUpdated\n7 days ago\n•\n447k\n•\n2.34k\ndeepseek-ai/DeepSeek-R1\nUpdated\n22 days ago\n•\n1.84M\n•\n11.4k\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n463\n463\nSesame CSM\n🌱\nConver

In [ ]:
def create_brochure(company_name: str, url: str) -> None:
    """
    Generates and displays a markdown brochure for a company.

    Parameters:
        company_name(str): Name of the company to create a brochure for.
        url(str): URL of the company's website.
    """
    response = groq.chat.completions.create(
        model=MODEL,
        temperature=2,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("Hugging Face", "https://huggingface.co/")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing', 'url': 'https://huggingface.co/pricing'}]}


**Hugging Face: The AI Community Building the Future 🤖💻**
============================================================

**Welcome to the Home of Machine Learning 🏠**
---------------------------------------------

Hugging Face, where the magic of AI happens! 🔮 We're a community-driven platform where machine learning enthusiasts collaborate on models, datasets, and applications. Our passion is to accelerate the development of AI and make it accessible to everyone.

**Explore the World of AI 🌐**
-----------------------------

* **1M+ Models**: Browse and discover the latest models, from conversational speech generation to advanced video generative models.
* **250k+ Datasets**: Access and share datasets for any ML task, from medical reasoning to natural language processing.
* **400k+ Applications**: Explore AI apps that will blow your mind, from virtual try-on to fast image relighting.

**Compute, Deploy, and Accelerate 💻**
-------------------------------------

* **Compute**: Deploy on optimized Inference Endpoints or update your Spaces applications to a GPU in a few clicks.
* **Enterprise**: Give your team the most advanced platform to build AI with enterprise-grade security, access controls, and dedicated support.

**Our Open Source ❤️**
-------------------------

We're building the foundation of ML tooling with the community. Our open-source projects include:

* **Transformers**: State-of-the-art ML for PyTorch, TensorFlow, JAX
* **Diffusers**: State-of-the-art Diffusion models in PyTorch
* **Safetensors**: Safe way to store/distribute neural network weights
* **Hub Python Library**: Python client to interact with the Hugging Face Hub
* **Tokenizers**: Fast tokenizers optimized for research & production
* **TRL**: Train transformers LMs with reinforcement learning
* **Transformers.js**: State-of-the-art ML running directly in your browser
* **smolagents**: Smol library to build great agents in Python
* **PEFT**: Parameter-efficient finetuning for large language models
* **Datasets**: Access & share datasets for any ML tasks
* **Text Generation Inference**: Serve language models with TGI optimized toolkit
* **Accelerate**: Train PyTorch models with multi-GPU, TPU, mixed precision

**Join the Hugging Face Family 👪**
-----------------------------------

We're a community-driven platform, and we're always looking for talented individuals to join our team. Check out our **Jobs** page for open positions.

**Stay Connected 📱**
---------------------

Follow us on **GitHub**, **Twitter**, **LinkedIn**, and **Discord** to stay up-to-date with the latest news, updates, and community discussions.

**Let's Build the Future of AI Together 🚀**
---------------------------------------------

Join the Hugging Face community today and start building the future of AI! 🤖💻

In [ ]:
create_brochure("Dunman High School", "https://www.dunmanhigh.moe.edu.sg/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.dunmanhigh.moe.edu.sg/about-dhs/principal-message/'}, {'type': 'about page', 'url': 'https://www.dunmanhigh.moe.edu.sg/about-dhs/mission-vision-motto-sch-spirit/'}, {'type': 'about page', 'url': 'https://www.dunmanhigh.moe.edu.sg/about-dhs/school-crest-song/'}, {'type': 'about page', 'url': 'https://www.dunmanhigh.moe.edu.sg/about-dhs/school-history/'}, {'type': 'contact page', 'url': 'https://www.dunmanhigh.moe.edu.sg/about-dhs/contact-us/'}, {'type': 'admissions page', 'url': 'https://www.dunmanhigh.moe.edu.sg/admissions/'}, {'type': 'careers page', 'url': 'https://www.dunmanhigh.moe.edu.sg/cshe-programme/cshe-careers-scholarships-and-higher-education/'}]}


**Welcome to Dunman High School 🎉**
=============================

**Where Learning Meets Fun 🤩**

Are you ready to embark on an exciting educational journey? 🚀 Look no further than Dunman High School, a Singapore Government Agency Website 🇸🇬! Our school is dedicated to providing a holistic education that prepares students for success in an ever-changing world 🌎.

**Our Mission 🎯**
----------------

At Dunman High School, we strive to nurture curious, creative, and critical thinkers who are passionate about learning and making a positive impact in society 🌟. Our mission is to provide a supportive and inclusive environment that fosters academic excellence, character development, and community engagement 🌈.

**Our Culture 🎉**
-----------------

We're a school that's all about balance 🤹‍♀️. We believe in providing a well-rounded education that combines academic rigor with extracurricular activities, community service, and character development 🌟. Our students are encouraged to explore their passions, develop their talents, and make lifelong friendships 👫.

**Our People 👥**
----------------

Meet our dedicated team of educators, administrators, and support staff who are committed to helping our students succeed 🙌. From our school leaders to our form teachers, we're a team that's passionate about making a difference in the lives of our students ❤️.

**Academic Excellence 📚**
-------------------------

We offer a range of academic programs that cater to different learning styles and interests 📊. From our Dunman High Programme to our Bicultural Studies Programme, Humanities Programme, Mathematics Programme, Science Programme, Aesthetics Programme, and Knowledge Skills Programme, we've got you covered 🎯.

**Student Development 🌱**
---------------------

We believe in providing a supportive environment that fosters student growth and development 🌈. Our student development programs include student leadership, talent development, co-curricular activities, sexuality education, physical education, and career guidance 🎯.

**Partnerships 🤝**
----------------

We're proud of our partnerships with our Parent Support Group, Dunman High School Alumni, and School Advisory Committee 🤝. Together, we're committed to providing a holistic education that prepares our students for success in an ever-changing world 🌎.

**Join Our Community 🌟**
-------------------------

Ready to be part of our vibrant community? 🎉 Join us on Facebook and Instagram to stay updated on the latest news, events, and achievements from Dunman High School 📱.

**Contact Us 📲**
-----------------

Have a question or inquiry? 🤔 Contact us through our general enquiries or school canteen advertisement 📲.

**Campus Life 🏫**
-----------------

Want to experience the best of campus life? 🎉 Join us for our Junior High (Sec) and Senior High (JC) programs, and discover a world of learning, growth, and fun 🌈!

**We Can't Wait to Meet You! 🎉**
-----------------------------------

Apply now and join the Dunman High School family 🎉! We can't wait to meet you and start this exciting educational journey together 🚀!

# 7. **Streaming Response**

- The typewriter animation :)

In [ ]:
def stream_brochure(company_name: str, url: str) -> None:
    """
    Generates and streams a markdown brochure for a company.

    Creates a streaming response from the LLM and updates the display
    in real-time as chunks of content arrive.

    Parameters:
        company_name: Name of the company to create a brochure for.
        url: URL of the company's website.
    """
    stream = groq.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("Hugging Face", "https://huggingface.co/")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'brand page', 'url': 'https://huggingface.co/brand'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}]}


**Hugging Face: The AI Community Building the Future 🤖💻**
============================================================

**Welcome to the Home of Machine Learning 🏠**
------------------------------------------------

Hugging Face is the ultimate platform where the machine learning community collaborates on models, datasets, and applications. With over 1 million models, 250,000 datasets, and 400,000 applications, we're the largest AI community in the world! 🌎

**What We Do 🤔**
-----------------

* **Collaborate**: Host and collaborate on unlimited public models, datasets, and applications.
* **Create**: Build and share your own ML projects with the world.
* **Discover**: Explore and learn from the largest collection of ML models and datasets.
* **Accelerate**: Move faster with our open-source stack and optimized compute solutions.

**Our Culture 🎉**
-------------------

* **Open-source**: We believe in building the foundation of ML tooling with the community.
* **Collaborative**: We're a platform where anyone can contribute, learn, and grow.
* **Innovative**: We're always pushing the boundaries of what's possible with AI.

**Our Customers 🤝**
-------------------

* **50,000+ organizations**: Including Ai2, Amazon, Google, Intel, Microsoft, and more!
* **Enterprise-grade**: We provide dedicated support, access controls, and security for our enterprise customers.

**Careers/Jobs 🎉**
-------------------

* **Join our team**: We're always looking for talented individuals to join our mission to build the future of AI.
* **Check out our job openings**: From software engineers to product managers, we have a role for you!

**Get Started 🚀**
-----------------

* **Sign up**: Create an account and start exploring our platform today!
* **Compute**: Deploy on optimized inference endpoints or update your spaces applications to a GPU in a few clicks.
* **Enterprise**: Give your team the most advanced platform to build AI with enterprise-grade security and support.

**Stay Connected 📱**
-------------------------

* **Follow us on social media**: GitHub, Twitter, LinkedIn, Discord, and more!
* **Join our community forum**: Discuss the latest ML trends and learn from others in the community.

**Let's Build the Future of AI Together! 🤝**
------------------------------------------------